<a href="https://colab.research.google.com/github/Federico6419/MachineLearningProject/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch import Tensor
from collections import deque
import config
from model import Model
import matplotlib.pyplot as plt

avarage_random = np.zeros(config.NUM_EPISODES_TEST) # avarage reward for each episode
avarage_table = np.zeros(config.NUM_EPISODES_TEST)
avarage_nn = np.zeros(config.NUM_EPISODES_TEST)
cum_reward_random = np.zeros(config.NUM_EPISODES_TEST) ## cumulative reward for each episode
cum_reward_table = np.zeros(config.NUM_EPISODES_TEST)
cum_reward_nn = np.zeros(config.NUM_EPISODES_TEST)

action_space = [
                (-1, 1, 0.2), (0, 1, 0.2), (1, 1, 0.2),
                (-1, 1,   0), (0, 1,   0), (1, 1,   0),               #(Steering Wheel, Gas, Break)
                (-1, 0, 0.2), (0, 0, 0.2), (1, 0, 0.2),               #Range -1~1 0~1 0~1
                (-1, 0,   0), (0, 0,   0), (1, 0,   0)
              ]

# define the model and load the weights
model = Model().to(config.DEVICE)
optimizer = optim.Adam(model.parameters(), lr=config.LR)
config.load_model(config.CHECKPOINT,model,optimizer)

#load the q table csv file
#Q = np.loadtxt('q_table_little_discretization2000.csv', delimiter=',')

## define the policy
def policy(pol,state):
    #if(pol == "random"):
    #    return env.action_space.sample()
    #elif(pol == "q_table"):
    #    return np.argmax(Q[state])
    #elif(pol == "nn"):
    prediction = model(torch.from_numpy(state.astype('float32')).to(config.DEVICE)).detach().cpu().numpy()
    action = action_space[np.argmax(prediction)]              #Select the action with the maximum predicted Q-Value

    return action


env = gym.make("CarRacing-v2", render_mode="human")
observation, info = env.reset()

###Testing
def test(types):
    c_reward = 0 #cumulative reward
    total_reward = 0
    episode = 0
    #for i in range(config.NUM_EPISODES_TEST):
    for i in range(10):
            observation, info = env.reset()
            for _ in range(250):
                env.render()
                #if(types == "random policy"):
                #    action = policy("random",observation)  # this is where you would insert your policy
                #elif(types == "q table policy"):
                #    state= config.discretize(observation)
                #    action = policy("q_table",state)
                #elif(types == "neural network policy"):
                action = policy("nn",observation)

                observation, reward, done, truncated, info = env.step(action)
                c_reward += reward


                if done or truncated:
                    total_reward += c_reward
                    break

            #if(types == "random policy"):
            #    cum_reward_random[episode] = c_reward # save the cum_reward for the relative episode
            #    avarage_random[episode]=total_reward/episode
            #elif(types == "q table policy"):
            #    cum_reward_table[episode] = c_reward
            #    avarage_table[episode]=total_reward/episode
            #elif(types == "neural network policy"):
            cum_reward_nn[episode] = c_reward
            avarage_nn[episode]=total_reward/episode

            print("The cumulative reward is:",c_reward)
            c_reward = 0 # reset the current cumulative reward
            print("Episode: ",episode)
            episode += +1

    print(f"Tests for {types} finished after {config.NUM_EPISODES_TEST} episodes")



#test("random policy")
#test("q table policy")
#test("neural network policy")

env.close()